## Evolution of possible outcomes during play

This notebook plots a graph that can be used to display the range of possible final scores for a game played with optimal strategy.

Each time a facedown card is revealed, the set of possible outcomes and relative probability of each is updated, the graph created by this notebook shows how the state space collapses.

In [ ]:
from cardgame import Game
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt
import pandas as pd
from random import choice

sns.set_style("darkgrid")
plt.style.use("dark_background")

We need to generate a game state to calculate the possible outcomes for, either use the cell below to generate a random state or run a cell containing one of the two suggested save states.

In [ ]:
# find a position with 14 cards remaining where 5 of them are facedown
# This is a computable position which still has quite high variability in the outcome
target_cards_remaining = 14
target_fd_cards_remaining = 5

while True:
    game = Game.deal()
    while game.legal_moves:
        game = game.random_move()
    moves_to_undo = max(0, len(game.moves) - 36 + target_cards_remaining)
    if not moves_to_undo:
        continue
    if len(game.undo(moves_to_undo).board.facedown_cards) == target_fd_cards_remaining:
        break
evaluation_state = game.undo(moves_to_undo)
evaluation_state

In [ ]:
save_state = "??8♥7♣A♥2♥K♣/A♣??5♥6♥6♣8♠/7♦5♠7♠2♦5♣6♠/K♦3♣????A♠7♥/3♥4♦A♦4♠2♣3♠/??4♥2♠8♦5♦3♦//K♠K♥8♣4♣6♦//94562364147289021770944"
evaluation_state = Game.load(save_state)

In [ ]:
save_state = "2♣3♥8♣A♠7♣K♠/2♥??8♥4♣??8♠/K♥K♣??4♥4♦7♥/6♦6♠4♠??A♦5♦/5♠3♦6♥3♣??5♣/K♦A♥8♦2♠2♦6♣//7♦7♠3♠A♣5♥//4989508823743703859608"
evaluation_state = Game.load(save_state)

We can now generate the plot data - this starts from the evaluation state, finding the possible score outcomes with optimal play and then makes the best move before repeating with the new state.

The cell below will record the evaluation at that point for each position in the game, with the information known to the players at that time. Whenever a move is made to a facedown card which has a random outcome we choose randomly between the possible facedown cards, and the information revealed by that move will mean the set of possible outcomes, and possible scores, can be updated, representing a collapse in the possible state space.

In [ ]:
game_evals, moves_fd_card_was_taken = [], []
fd_card_count = len(evaluation_state.board.facedown_cards)
move_index_base = len(evaluation_state.moves)
negamax_inversion = False
known_moves = []

while evaluation_state.legal_moves:
    # If the move was to a faceup card the evaluation remains the same, so as long as we know the best move we don't need to re-evaluate
    if (
        game_evals
        and known_moves
        and (len(evaluation_state.board.facedown_cards) == fd_card_count)
    ):
        score_fmap = game_evals[-1]
    else:
        full_eval = evaluation_state.evaluate()
        known_moves = full_eval["Deterministic optimal moves"]
        # The score is must be inverted on alternate moves to retain a consistent perspective for the score difference
        score_fmap = (
            -full_eval["Evaluation"] if negamax_inversion else full_eval["Evaluation"]
        )

    game_evals.append(score_fmap)
    negamax_inversion = not negamax_inversion

    # Record which moves were facedown moves so we can plot the information later
    if len(evaluation_state.board.facedown_cards) != fd_card_count:
        fd_card_count -= 1
        moves_fd_card_was_taken.append(len(evaluation_state.moves))

    # Make the best move to update the state being evaluated
    best_move = known_moves[0]
    known_moves = known_moves[1:]
    # The evaluation format returns the card to take, not the position to move to so we map that back to the position
    if hasattr(best_move, "rank"):
        best_move = next(
            move[0].marker
            for move in evaluation_state.all_moves()
            if len(move) == 1 and move[0].taken_card == best_move
        )
    evaluation_state = choice(evaluation_state.move(*best_move))

# Add the evaluation for the terminal state, and display the values
full_eval = evaluation_state.evaluate()
game_evals.append(
    -full_eval["Evaluation"] if negamax_inversion else full_eval["Evaluation"]
)
if len(evaluation_state.board.facedown_cards) != fd_card_count:
    moves_fd_card_was_taken.append(len(evaluation_state.moves))
game_evals, moves_fd_card_was_taken

With the evaluations at each point calculated we can plot a graph - there are many ways to represent this data, I use a heatmap with some extra lines.

If you are trying to follow the code, know that the data for the lineplot has to be plotted on the heatmap's categorical axes which requires some data manipulation.

You can use this graph to understand how likely the final outcome was calculated to be at each moment it the game history.

In [ ]:
# Setup a plot
fig, ax = plt.subplots(figsize=(13, 5))
ax.grid(visible=None)

# Format the score data to name indexes and include missing values
df = pd.DataFrame(game_evals).T.sort_index(ascending=False)
df.columns = df.columns = pd.RangeIndex(
    start=move_index_base, stop=move_index_base + len(game_evals), name="Move"
)
df.columns = pd.RangeIndex(
    start=move_index_base, stop=move_index_base + len(game_evals), name="Move"
)
df = df.reindex(
    pd.RangeIndex(start=max(df.index), stop=min(df.index) - 1, step=-1, name="Score")
)

# Use a heatmap to show how likely each outcome is evaluated as being from each point in the game history
sns.heatmap(
    df / df.sum(),
    cmap=mpl.colors.ListedColormap(
        mpl.colormaps["magma"].colors[16:]
    ),  # Drop the darkest part of the colormap to more clearly separate possible/impossible values
    linewidths=0.6,
    linecolor="#151515",
    cbar_kws={"pad": 0.01, "label": "Probability"},
    vmin=0,
    vmax=1,
    ax=ax,
    annot=True,
    fmt=".1%",
    annot_kws={"fontsize": "x-small"},
)

# Add marks to highlight where facedown moves were made, and what the mean score difference is for the possible outcomes from each position
ax.vlines(
    x=[mn - move_index_base for mn in moves_fd_card_was_taken],
    ymin=0,
    ymax=df.shape[0],
    colors="#585",
    ls="--",
    lw=2,
    alpha=0.5,
    label="Facedown move",
)
sns.lineplot(
    x=[i + 0.5 for i in range(len(game_evals))],
    y=[
        df.index.max() + 0.5 - (sum(k * v for k, v in scr.items()) / scr.multiplicity)
        for scr in game_evals
    ],
    label="Mean Score",
    linewidth=2,
    ax=ax,
)

plt.show()

You can now browse through the game states at each move and learn what the best move was in each position, and which facedown cards each player obtained and understand how that impacted the score.

If you want to see what the full evaluation was at that point, including information about the other moves clicking `Show evaluations details` will calculate and display the full information

In [ ]:
import ipywidgets as w
from cardgame import ProbEval

move_number = w.BoundedIntText(
    description="Move number",
    value=df.columns.max(),
    min=df.columns.min(),
    max=df.columns.max(),
    style={"description_width": "initial"},
)
full_details_bt = w.Button(
    description="Show evaluation details",
    button_style="primary",
    layout=w.Layout(width="15em"),
)
out = w.Output()
out2 = w.Output()


@out.capture(clear_output=True, wait=True)
def display_state(event):
    state = evaluation_state.undo(len(evaluation_state.moves) - event["new"])
    print("Position string:", state.save())
    if state.board.facedown_cards:
        print(
            f"Facedown cards in this position: {', '.join(str(x) for x in state.board.facedown_cards)}"
        )
    else:
        print()
    display(state)
    out2.clear_output()


@out2.capture(clear_output=True, wait=True)
def full_eval_details(event):
    state = evaluation_state.undo(len(evaluation_state.moves) - move_number.value)
    state_eval = state.evaluate()
    fig, ax = plt.subplots(figsize=(6, 4))
    (
        pd.DataFrame.from_dict(
            state_eval["Evaluation"], orient="index", columns=["Frequency"]
        )
        .reindex(pd.RangeIndex(min(temp), max(temp) + 1))
        .reset_index()
        .rename({"index": "Score"}, axis=1)
        .plot.bar(
            x="Score",
            y="Frequency",
            grid=False,
            title="Score frequencies",
            legend=False,
            width=1,
            ax=ax,
        )
    )
    plt.show()
    print(
        f"These score frequencies give a static evaluation of {str(state_eval['Evaluation'].eval)}"
    )

    if state_eval["Deterministic optimal moves"]:
        print("\nThe best move order from this position (that is deterministic) is to:")
        for move in state_eval["Deterministic optimal moves"]:
            if hasattr(move, "rank"):
                print(f"\t take the {str(move)}")
            else:
                print(
                    f"\t take the facedown card in row {move[0] + 1}, columns {move[1] + 1}"
                )

    if state_eval["Known info for other branches"]:
        print("\nKnown information for all moves from this position:")
        for move, move_details in state_eval["Known info for other branches"].items():
            if hasattr(move_details, "multiplicity"):
                print(f"\tTaking the {str(state.board[move[0]][move[1]])}:")
                print(
                    f"\t\tEvaluation is at least {str(move_details.lower_bound.eval)}"
                )
                print(f"\t\t{str(move_details)}")
            else:
                print(
                    f"\tTaking the facedown card at row {move[0] + 1}, column {move[1] + 1}"
                )
                print(
                    f"\t\tThis move has an overall evaluation of at least {str(ProbEval.combine(list(move_details.values())).lower_bound.eval)}"
                )
                for res, res_info in sorted(
                    move_details.items(),
                    key=lambda x: x[1].lower_bound.eval,
                    reverse=True,
                ):
                    print(f"\t\t{str(res)}: {str(res_info)}")


display_state({"new": len(evaluation_state.moves)})
move_number.observe(display_state, "value")
full_details_bt.on_click(full_eval_details)
w.VBox([w.HBox([move_number, full_details_bt]), w.HBox([out, out2])])